# NIMA Model Evaluation
This notebook evaluates image quality using the Neural Image Assessment (NIMA) model. 
It computes technical and aesthetic scores for a set of images.


## Install Dependencies

In [ ]:
# Install necessary packages
!pip install tensorflow
!pip install pillow
!pip install pandas
!git clone https://github.com/idealo/image-quality-assessment.git

In [1]:
import sys

# Create __init__.py in the evaluater and handlers folders
open('image-quality-assessment/src/evaluater/__init__.py', 'a').close()
open('image-quality-assessment/src/handlers/__init__.py', 'a').close()
open('image-quality-assessment/src/utils/__init__.py', 'a').close()

sys.path.append('image-quality-assessment/src')

In [18]:
import os
import glob
import pandas as pd
from PIL import Image
from utils.utils import calc_mean_score, save_json
from handlers.model_builder import Nima
from handlers.data_generator import TestDataGenerator

## Load Images from Directory

In [39]:
def image_dir_to_json(img_dir, img_type='jpg'):
    """
    Convert images in a directory to a JSON format.

    Args:
        img_dir (str): The directory path where the images are located.
        img_type (str, optional): The image file extension. Defaults to 'jpg'.

    Returns:
        list: A list of dictionaries, where each dictionary represents an image with its ID.

    """
    img_paths = glob.glob(os.path.join(img_dir, '*.'+img_type))

    samples = []
    for img_path in img_paths:
        img_id = os.path.basename(img_path).split('.')[0]
        samples.append({'image_id': img_id})

    return samples

In [40]:
# function converts all the images in the image directory to jpg and saves them in a new directory
def convert_images(image_dir,output_dir):
    """
    Convert images in a directory to PNG format.

    Args:
        image_dir (str): The directory path where the images are located.

    """
    img_paths = glob.glob(os.path.join(image_dir, '*'))

    for img_path in img_paths:
        try:
            img_id = os.path.basename(img_path).split('.')[0]
            img = Image.open(img_path)
            img.save(os.path.join(output_dir, img_id+'.png'))
        except:
            print(f'Error converting {img_path}')

## Convert the Images to PNG, if neccessary

In [41]:
image_dir = # TODO change this to the path of the images you want to evaluate
output_dir = # TODO change this to the path where you want to save the converted images

os.makedirs(output_dir, exist_ok=True)

convert_images(image_dir,output_dir)

Error converting /Volumes/tus6/b50/Hiwis/04_Leo/Amelie_Meta_Pictures/Thumbs.db


In [42]:
input_dir = #TODO change this to the path where the converted images are saved

samples = image_dir_to_json(input_dir, 'png')

## Load Model Weights and Make Predictions

In [43]:
# build model and load weights
nima = Nima('MobileNet', weights=None)
nima.build()
data_generator = TestDataGenerator(samples, input_dir, 64, 10, nima.preprocessing_function(),img_format='png')

weigths_file_technical = 'image-quality-assessment/models/MobileNet/weights_mobilenet_technical_0.11.hdf5'
weigths_file_aesthetic = 'image-quality-assessment/models/MobileNet/weights_mobilenet_aesthetic_0.07.hdf5'

In [44]:
technical=True
aesthetic=True

if technical:
    nima.nima_model.load_weights(weigths_file_technical)

    predictions = nima.nima_model.predict(
        data_generator,
        batch_size=10,
        verbose=1)
    
    # calc mean scores and add to samples
    for i, sample in enumerate(samples):
        sample['nima_technical'] = calc_mean_score(predictions[i])

if aesthetic:
    nima.nima_model.load_weights(weigths_file_aesthetic)

    predictions = nima.nima_model.predict(
        data_generator,
        batch_size=10,
        verbose=1)
    
    # calc mean scores and add to samples
    for i, sample in enumerate(samples):
        sample['nima_aesthetic'] = calc_mean_score(predictions[i])

/Users/leonardkinzinger/Desktop/NIMA/.venv/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 935ms/stepWARNING:tensorflow:5 out of the last 12 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x2d4664ee0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 673ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 564ms/step


## Save Results as a Pandas Dataframe

In [45]:
df = pd.DataFrame(samples)
df

,image_id,nima_technical,nima_aesthetic
0,214_Choi_02_human_woman_full_color,4.539200,5.429706
1,142_Wan_and_Jiang_01_a_virtual_woman_fullbody_...,5.274675,4.360690
2,121_Nissen_20_human_woman_upperbody_bw,5.214574,4.584579
3,151_Yang_01_a_human_woman_faceshot_color,5.999182,4.206086
4,121_Nissen_08_virtual_woman_fullbody_bw,5.719997,5.174316
...,...,...,...
149,53_Ozdemir_02_a_virtual_woman_upperbody_color,5.660220,5.461650
150,31_Stein_01_b_virtual_woman_fullstimuli_color,5.554849,5.102993
151,92_Li_01_human_woman_fullprofile_color,4.558420,4.242528
152,142_Wan_and_Jiang_06_b_human_woman_fullbody_color,4.929629,4.630395


In [46]:
output_path = #TODO change this to the path where you want to save the output csv file
df.to_csv(output_path, index=False)